### Your name:

<pre> Priya Kalyanakrishnan</pre>


### NOTE: 

<pre> 
tensorflow                 1.10.0           eigen_py35h38c8211_0

tensorflow-base            1.10.0           eigen_py35h45df0d8_0

tensorflow-gpu        -    1.12.0       -             pip

jupyter             -      1.0.0         -           py35_7

jupyter-tensorboard  -     0.1.8          -           pip
    
jupyter_client        -    5.2.3           -         py35_0

jupyter_console        -   5.2.0            -        py35_1

jupyter_core        -      4.4.0          -          py35_0

jupyterlab       -         0.34.9          -         py35_0

jupyterlab_launcher   -    0.13.1            -       py35_0

anaconda-client     -      1.7.2            -        py35_0

anaconda-navigator   -     1.9.2           -         py35_0

anaconda-project      -    0.8.2           -         py35_0

conda        -             4.5.11         -          py35_0

conda-build      -         3.15.1        -           py35_0

conda-env         -        2.6.0        -        h36134e3_1

msys2-conda-epoch    -     20160418       -               1

pip           -            10.0.1            -       py35_0

pip          -             18.1               -       pip


</pre>

### Collaborators:
### References:
https://github.com/ageron/handson-ml/blob/master/09_up_and_running_with_tensorflow.ipynb

09_up_and_running_with_tensorflow_modified     ____         *//Jupyter Notebook//*

Textbook: Hands-On Machine Learning with Scikit-Learn & Tensorflow - O'Reilly, Aurelien Geron

https://www.tensorflow.org/api_docs/python/tf/losses/absolute_difference

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
# to make this notebook's output stable across runs
np.random.seed(123)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12


### TensorFlow

Q1. When is a variable initialized? When is it destroyed?

When the initializer function is called, the variable is initialized and the variable is destroyed when the session ends.


Q2. What is the difference between a placeholder and a variable?

Variable holds a value, data, along with model parameters when initialized. Placeholder holds information about the tensor, type and shape used in feed_dict. 

Q3. How many times does reverse-mode autodiff need to traverse the graph in order to compute the gradients of the cost function with regards to 10 variables? What about forward-mode autodiff? And symbolic differentiation?

Twice for autodiff.

Ten times for forward-mode.

None because symbolic differentiation does not transverse. 

https://github.com/ageron/handson-ml/blob/1a1d5149a28cd63cb48dadaeaad3a00c104ecdd0/09_up_and_running_with_tensorflow.ipynb

Q4. Implement Logistic Regression with Mini-batch Gradient Descent using TensorFlow. Train it and evaluate it on the moons dataset (introduced in Chapter 5). Try adding all the bells and whistles:

- Define the graph within a logistic_regression() function that can be reused easily.

- Save checkpoints using a Saver at regular intervals during training, and save the final model at the end of training.

- Restore the last checkpoint upon startup if training was interrupted.

- Define the graph using name scopes so the graph looks good in TensorBoard.

- Add summaries to visualize the learning curves in TensorBoard.

- Try tweaking some hyperparameters such as the learning rate or the mini-batch size and look at the shape of the learning curve.

## Q4.

Created a function to reset graph for tensorflow data accumulation

In [2]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

importing moon dataset

In [3]:
from sklearn.datasets import make_moons

m = 1000
X_moons, y_moons = make_moons(m, noise=0.1, random_state=42)

Specifically creating numpy values from moonset

In [4]:

X_moons_with_bias = np.c_[np.ones((m, 1)), X_moons]

In [5]:

y_moons_column_vector = y_moons.reshape(-1, 1)

Created Test and train variables

In [6]:
test_ratio = 0.2
test_size = int(m * test_ratio)
X_train = X_moons_with_bias[:-test_size]
X_test = X_moons_with_bias[-test_size:]
y_train = y_moons_column_vector[:-test_size]
y_test = y_moons_column_vector[-test_size:]

Created function to create batches

In [7]:

def random_batch(X, y, batch_size):
    rnd_indices = np.random.randint(0, len(X), batch_size)
    X_batch = X[rnd_indices]
    y_batch = y[rnd_indices]
    return X_batch, y_batch

Variable for initializer

In [8]:
init = tf.global_variables_initializer()

Function created for logistic regression

In [9]:
def logistic_regression(X, y, initializer=None, seed=42, learning_rate=0.01):
    n_inputs_including_bias = int(X.get_shape()[1])
    with tf.name_scope("logistic_regression"):
        with tf.name_scope("model"):
            if initializer is None:
                initializer = tf.random_uniform([n_inputs_including_bias, 1], -1.0, 1.0, seed=seed)
            theta = tf.Variable(initializer, name="theta")
            logits = tf.matmul(X, theta, name="logits")
            y_proba = tf.sigmoid(logits)
        with tf.name_scope("train"):
            loss = tf.losses.log_loss(y, y_proba, scope="loss")
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
            training_op = optimizer.minimize(loss)
            loss_summary = tf.summary.scalar('log_loss', loss)
        with tf.name_scope("init"):
            init = tf.global_variables_initializer()
        with tf.name_scope("save"):
            saver = tf.train.Saver()
    return y_proba, loss, training_op, loss_summary, init, saver

Function created to place the tensorflow files to a directory

In [10]:
from datetime import datetime

def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

Variables created for the final stage of processing

In [11]:
X_train_enhanced = np.c_[X_train,
                         np.square(X_train[:, 1]),
                         np.square(X_train[:, 2]),
                         X_train[:, 1] ** 3,
                         X_train[:, 2] ** 3]
X_test_enhanced = np.c_[X_test,
                        np.square(X_test[:, 1]),
                        np.square(X_test[:, 2]),
                        X_test[:, 1] ** 3,
                        X_test[:, 2] ** 3]

Imported modules for precision and recall score

In [12]:
from sklearn.metrics import precision_score, recall_score

## Final evaluation

The code will automatically place file in the directory accordingly.

In [13]:
from scipy.stats import reciprocal

n_search_iterations = 10

for search_iteration in range(n_search_iterations):
    batch_size = np.random.randint(1, 100)
    learning_rate = reciprocal(0.0001, 0.1).rvs(random_state=search_iteration)

    n_inputs = 2 + 4
    logdir = log_dir("logreg")
    
    print("Iteration", search_iteration)
    print("  logdir:", logdir)
    print("  batch size:", batch_size)
    print("  learning_rate:", learning_rate)
    print("  training: ", end="")

    reset_graph()

    X = tf.placeholder(tf.float32, shape=(None, n_inputs + 1), name="X")
    y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

    y_proba, loss, training_op, loss_summary, init, saver = logistic_regression(
        X, y, learning_rate=learning_rate)

    file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

    n_epochs = 10001
    n_batches = int(np.ceil(m / batch_size))

    final_model_path = "./my_logreg_model_%d" % search_iteration

    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(n_epochs):
            for batch_index in range(n_batches):
                X_batch, y_batch = random_batch(X_train_enhanced, y_train, batch_size)
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            loss_val, summary_str = sess.run([loss, loss_summary], feed_dict={X: X_test_enhanced, y: y_test})
            file_writer.add_summary(summary_str, epoch)
            if epoch % 500 == 0:
                print(".", end="")

        saver.save(sess, final_model_path)

        print()
        y_proba_val = y_proba.eval(feed_dict={X: X_test_enhanced, y: y_test})
        y_pred = (y_proba_val >= 0.5)
        
        print("  precision:", precision_score(y_test, y_pred))
        print("  recall:", recall_score(y_test, y_pred))

Iteration 0
  logdir: tf_logs/logreg-run-20181127195856/
  batch size: 67
  learning_rate: 0.00443037524522
  training: .....................
  precision: 0.979797979798
  recall: 0.979797979798
Iteration 1
  logdir: tf_logs/logreg-run-20181127200011/
  batch size: 48
  learning_rate: 0.00178264971514
  training: .....................
  precision: 0.969696969697
  recall: 0.969696969697
Iteration 2
  logdir: tf_logs/logreg-run-20181127200148/
  batch size: 75
  learning_rate: 0.00203228544324
  training: .....................
  precision: 0.969696969697
  recall: 0.969696969697
Iteration 3
  logdir: tf_logs/logreg-run-20181127200256/
  batch size: 86
  learning_rate: 0.00449152382514
  training: .....................
  precision: 0.979797979798
  recall: 0.979797979798
Iteration 4
  logdir: tf_logs/logreg-run-20181127200358/
  batch size: 87
  learning_rate: 0.0796323472178
  training: .....................
  precision: 0.980198019802
  recall: 1.0
Iteration 5
  logdir: tf_logs/logreg-

Q5. Similar to the linear regression implementation in class, write a lasso regression implementation. Use the same dataset, and choose a value for the penalty $\alpha$:

Using a Saver at regular intervals during training, and save the final model at the end of training.

Restore the last checkpoint upon startup if training was interrupted.

Define the graph using name scopes so the graph looks good in TensorBoard.

Add summaries to visualize the learning curves in TensorBoard.

Try tweaking some hyperparameters such as the learning rate or the mini-batch size and look at the shape of the learning curve.

## Q5.

The following steps are the same as the previous codes above. They are placed there again in case the above codes were not processed.

In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
# to make this notebook's output stable across runs
np.random.seed(123)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

In [15]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

Creating a pandas dataframe from moons dataset

In [16]:
import numpy as np
import pandas as pd

reset_graph()

moons_df = pd.DataFrame(data= [X_moons, y_moons])

Constants created with tensorflow function

In [17]:
X = tf.constant(X_moons_with_bias, dtype=tf.float32, name="X")
y = tf.constant(y_moons_column_vector.reshape(-1, 1), dtype=tf.float32, name="y")

Theta value function processed as per textbook

In [18]:
XT = tf.transpose(X)
print("XT.shape = ", XT.shape)


theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    T_value= sess.run(y)
    print("y =", T_value)
    theta_value = theta.eval()

XT.shape =  (3, 1000)
y = [[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 1.]
 [ 1.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 

In [19]:
theta_value

array([[ 0.57189882],
       [ 0.17028242],
       [-0.6339373 ]], dtype=float32)

Lasso Regression produced the very similar results as the above code

In [20]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=0.000001)
lasso_reg.fit(X_moons, y_moons_column_vector.reshape(-1, 1))
print(lasso_reg.intercept_.reshape(-1, 1), lasso_reg.coef_.T)

[[ 0.571897]] [ 0.1702833 -0.6339326]


Numpy function also produced similar results

In [21]:
X = X_moons_with_bias
y = y_moons_column_vector.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)

[[ 0.57189852]
 [ 0.17028251]
 [-0.63393714]]


Created another directory for the Lasso Regression tensorflow files

In [22]:
from datetime import datetime

def log_dir2(prefix=""):
    now = datetime.utcnow().strftime("%m%d%Y%H%M%S")
    root_logdir = "tf_Lasso_R___weights+2.3"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

Function for lasso Regression

In [23]:
def lasso_regression(X, y, initializer, seed, learning_rate):
    tf.set_random_seed(seed)
    n_inputs_including_bias = int(X.get_shape()[1])
    with tf.name_scope("lasso_regression"):
        with tf.name_scope("model"):
            if initializer is None:
                initializer = tf.random_uniform([n_inputs_including_bias, 1], -1.0, 1.0, seed=seed)
            theta = tf.Variable(initializer, name="theta")
            logits = tf.matmul(X, theta, name="logits")
            y_proba = tf.sigmoid(logits)
        with tf.name_scope("train"):
            loss = tf.losses.absolute_difference(y, y_proba, weights=2.3, scope="loss")
            optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
            training_op = optimizer.minimize(loss)
            loss_summary = tf.summary.scalar('lasso_loss', loss)
        with tf.name_scope("init"):
            init = tf.global_variables_initializer()
        with tf.name_scope("save"):
            saver = tf.train.Saver()
    return y_proba, loss, training_op, loss_summary, init, saver

## Final evaluation

Similar to the Logistic Regression's final processing code for tensorflow files, replaced with Lasso Regression.

In [24]:
from scipy.stats import reciprocal

n_search_iterations = 10

for search_iteration in range(n_search_iterations):
    batch_size = np.random.randint(1, 100)
    learning_rate = reciprocal(0.0001, 0.1).rvs(random_state=search_iteration)

    n_inputs = 2 + 4
    logdir2 = log_dir2("lasso_regression")
    n_epochs = 10001
    n_batches = int(np.ceil(m / batch_size))

    print("Iteration", search_iteration)
    print("  logdir:", logdir2)
    print("  batch size:", batch_size)
    print("  learning_rate:", learning_rate)
    print("  training: ", end="")

    reset_graph()

    X = tf.placeholder(tf.float32, shape=(None,  n_inputs + 1), name="X")

    y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

                   
    y_proba, loss, training_op, loss_summary, init, saver = lasso_regression(X, y, initializer=None, seed=42, learning_rate=learning_rate)
                   
    file_writer = tf.summary.FileWriter(logdir2, tf.get_default_graph())

    final_model_path = "./my_lassoreg_model_%d" % search_iteration
    
    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(n_epochs):
            for batch_index in range(n_batches):
                X_batch, y_batch = random_batch(X_train_enhanced, y_train, batch_size)
                sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            loss_val, summary_str = sess.run([loss, loss_summary], feed_dict={X: X_test_enhanced, y: y_test})
            file_writer.add_summary(summary_str, epoch)
            if epoch % 500 == 0:
                print(".", end="")


        saver.save(sess, final_model_path)

        print()
        y_proba_val = y_proba.eval(feed_dict={X: X_test_enhanced, y: y_test})
        y_pred = (y_proba_val >= 0.5)
        
        print("  precision:", precision_score(y_test, y_pred))
        print("  recall:", recall_score(y_test, y_pred))

Iteration 0
  logdir: tf_Lasso_R___weights+2.3/lasso_regression-run-11272018201125/
  batch size: 52
  learning_rate: 0.00443037524522
  training: .....................
  precision: 0.979797979798
  recall: 0.979797979798
Iteration 1
  logdir: tf_Lasso_R___weights+2.3/lasso_regression-run-11272018201253/
  batch size: 73
  learning_rate: 0.00178264971514
  training: .....................
  precision: 0.969696969697
  recall: 0.969696969697
Iteration 2
  logdir: tf_Lasso_R___weights+2.3/lasso_regression-run-11272018201356/
  batch size: 6
  learning_rate: 0.00203228544324
  training: .....................
  precision: 0.979797979798
  recall: 0.979797979798
Iteration 3
  logdir: tf_Lasso_R___weights+2.3/lasso_regression-run-11272018202512/
  batch size: 24
  learning_rate: 0.00449152382514
  training: .....................
  precision: 0.979797979798
  recall: 0.979797979798
Iteration 4
  logdir: tf_Lasso_R___weights+2.3/lasso_regression-run-11272018202806/
  batch size: 75
  learning_r

## Conclusion

I found that changing the weight of the lasso regression function in tensorflow (absolute_difference) could in fact be very similar to logistic regression. Weights 0, 1, 1.5, and 2.3 were used and 2.3 seemed more in the shape of logistic regression curves while any lower number seemed to be more of a linear line than a logistical curve. The learning curve also impacted the precision and recall scores of both functions.